In [1]:
from pyopenms import *
import pyopenms as pms
import pandas as pd
import numpy as np
import os
import glob

input_feature_files = glob.glob('results/consensus/interim/MapAligner*.featureXML')

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

use_centroid_rt= False
use_centroid_mz= True
protein_ids = []
peptide_ids= []



mapper = pms.IDMapper()

input_mzml_files= glob.glob("results/interim(*mzML")

for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)

    for fmap in feature_maps: 
        if os.path.basename(fmap.getMetaValue('spectra_data')[0].decode()) == os.path.basename(filename):

#            peptide_ids = []
#            protein_ids = []
#            protein_id = ProteinIdentification()
#            protein_id.setIdentifier(fmap.getMetaValue('spectra_data')[0].decode())
#            protein_ids.append(protein_id)
#            peptide_id = PeptideIdentification()

            mapper.annotate(fmap, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
            
            proteins = fmap.getProteinIdentifications()

            fmap.setProteinIdentifications(proteins)

            for index_protein, protein in enumerate(proteins):
                protein.setIdentifier(fmap.getMetaValue('spectra_data')[0])
                print(protein.getIdentifier)
            
            for index_feature, feature in enumerate(fmap):
                peptide_ids = feature.getPeptideIdentifications()
                for index_pep, pep in enumerate(feature.getPeptideIdentifications()):
                    pep.setIdentifier(fmap.getMetaValue('spectra_data')[0])
            feature.setPeptideIdentifications(peptide_ids)
                    #print(pep.getIdentifier())
        featureidx_file = os.path.join("results", "", "consensus", "", "interim", "", 'IDMapper_' + os.path.basename(fmap.getMetaValue('spectra_data')[0].decode())[19:-5] +".featureXML")
        FeatureXMLFile().store(featureidx_file, fmap)

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail
